# Silicon (Si) Complete DFT Study with SHALOM

This notebook demonstrates a complete computational materials science workflow
for diamond-cubic silicon using the SHALOM framework with Quantum ESPRESSO.

## What We Will Compute

| Step | Analysis | Method | Time Estimate |
|------|----------|--------|---------------|
| 1 | Crystal symmetry | spglib | < 1 sec |
| 2 | XRD pattern | pymatgen | < 1 sec |
| 3 | ecutwfc convergence | 5 SCF runs | ~5 min |
| 4 | k-point convergence | 5 SCF runs | ~5 min |
| 5 | Full workflow (vc-relax → scf → bands → nscf → dos) | StandardWorkflow | ~10 min |
| 6 | Electronic analysis | Band gap, effective mass | < 1 sec |
| 7 | Combined band+DOS plot | Publication figure | < 1 sec |
| 8 | Phonon dispersion + thermal | Finite displacement + phonopy | ~5 min |

**Total: ~25–35 minutes**

## Prerequisites

- conda env `shalom-env` with Python 3.11
- Quantum ESPRESSO (pw.x, dos.x) installed in WSL
- SSSP pseudopotentials: `python -m shalom setup-qe --elements Si --download`

In [ ]:
import os
import time
import warnings

import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

from ase.build import bulk

warnings.filterwarnings('ignore', category=DeprecationWarning)

# === Configuration (adjust for your environment) ===
PSEUDO_DIR = r"C:\Users\Sejong\pseudopotentials"  # path to SSSP UPF files
OUTPUT_ROOT = os.path.expanduser("~/Desktop/shalom-tutorials/si_study")
os.makedirs(OUTPUT_ROOT, exist_ok=True)

WSL = True      # Set False for native Linux/macOS
NPROCS = 2      # MPI processes for pw.x

print(f"Pseudo dir:      {PSEUDO_DIR}")
print(f"Output directory: {OUTPUT_ROOT}")
print(f"WSL mode: {WSL}")
print(f"MPI processes: {NPROCS}")

## 1. Structure Creation

Silicon crystallizes in the diamond cubic structure (space group Fd¯m, #227).
We use ASE's `bulk()` function to create the 2-atom primitive cell
with the experimental lattice parameter a = 5.431 Å.

In [ ]:
si = bulk("Si", "diamond", a=5.431)

print(f"Formula:     {si.get_chemical_formula()}")
print(f"Atoms:       {len(si)}")
print(f"Cell (\u00c5):")
for i, v in enumerate(si.cell):
    print(f"  a{i+1} = [{v[0]:8.4f}, {v[1]:8.4f}, {v[2]:8.4f}]")
print(f"Volume:      {si.get_volume():.2f} \u00c5\u00b3")
print(f"Positions (fractional):")
for sym, pos in zip(si.get_chemical_symbols(), si.get_scaled_positions()):
    print(f"  {sym}: ({pos[0]:.4f}, {pos[1]:.4f}, {pos[2]:.4f})")

## 2. Symmetry Analysis (spglib)

Before running any DFT calculations, we characterize the crystal symmetry.
SHALOM wraps spglib to determine the space group, point group, Wyckoff
positions, and crystal system.

In [ ]:
from shalom.analysis import analyze_symmetry, is_spglib_available

if is_spglib_available():
    sym = analyze_symmetry(si)
    print("=== Crystal Symmetry ===")
    print(f"Space group:    {sym.space_group_symbol} (#{sym.space_group_number})")
    print(f"Crystal system: {sym.crystal_system}")
    print(f"Point group:    {sym.point_group}")
    print(f"Lattice type:   {sym.lattice_type}")
    print(f"Hall symbol:    {sym.hall_symbol}")
    print(f"Is primitive:   {sym.is_primitive}")
    print(f"Symmetry ops:   {sym.n_operations}")
    print(f"Wyckoff:        {sym.wyckoff_letters}")
else:
    print("spglib not installed. Run: pip install spglib")

## 3. X-ray Diffraction Pattern (pymatgen)

Simulated powder XRD pattern using Cu Kα radiation (λ = 1.5406 Å).
This is a pre-DFT analysis that fingerprints the crystal structure.

In [ ]:
from shalom.analysis import calculate_xrd, is_xrd_available
from shalom.plotting import XRDPlotter

if is_xrd_available():
    xrd_result = calculate_xrd(si, wavelength="CuKa", two_theta_range=(10, 90))

    print(f"Number of peaks: {xrd_result.n_peaks}")
    print(f"Wavelength: {xrd_result.wavelength} ({xrd_result.wavelength_angstrom:.4f} \u00c5)")
    print(f"\nTop 5 peaks:")
    print(f"{'2\u03b8 (deg)':>12}  {'Intensity':>10}  {'(h k l)':>10}  {'d (\u00c5)':>8}")
    print("-" * 50)
    order = np.argsort(xrd_result.intensities)[::-1]
    for i in order[:5]:
        hkl = xrd_result.hkl_indices[i]
        print(f"{xrd_result.two_theta[i]:12.2f}  {xrd_result.intensities[i]:10.1f}  "
              f"({hkl[0]:2d} {hkl[1]:2d} {hkl[2]:2d})  {xrd_result.d_spacings[i]:8.4f}")

    xrd_path = os.path.join(OUTPUT_ROOT, "si_xrd.png")
    fig = XRDPlotter(xrd_result).plot(
        output_path=xrd_path,
        title="Si (diamond) \u2014 Simulated XRD (Cu K\u03b1)",
    )
    plt.show()
    print(f"\nSaved: {xrd_path}")
else:
    print("pymatgen not installed. Run: pip install pymatgen")

## 4. Convergence Tests

Before production calculations, we determine the optimal parameters:
1. **ecutwfc convergence**: Plane-wave cutoff energy (Ry)
2. **k-point convergence**: Monkhorst-Pack grid density

Each test runs a series of SCF calculations and identifies the parameter value
where total energy converges to within 1 meV/atom of the reference (highest value).

### 4a. ecutwfc Convergence

SSSP Efficiency v1.3.0 recommends ecutwfc = 30 Ry for Si.
We test a range from 20 to 60 Ry with a fixed k-grid [4,4,4].

**Estimated time: ~5 minutes (5 SCF calculations)**

In [ ]:
from shalom.workflows import CutoffConvergence
from IPython.display import display, Image

ecut_values = [20, 30, 40, 50, 60]  # Ry
ecut_dir = os.path.join(OUTPUT_ROOT, "convergence_ecutwfc")

print(f"Testing ecutwfc values: {ecut_values} Ry")
print(f"Fixed k-grid: [4, 4, 4]")

t0 = time.time()
conv_ecut = CutoffConvergence(
    atoms=si,
    output_dir=ecut_dir,
    values=ecut_values,
    kgrid=[4, 4, 4],
    nprocs=NPROCS,
    wsl=WSL,
    pseudo_dir=PSEUDO_DIR,
    timeout=600,
)
result_ecut = conv_ecut.run()
elapsed = time.time() - t0

print(f"\n{result_ecut.summary()}")
print(f"Elapsed: {elapsed:.1f} s")

# Per-value results table
print(f"\n{'ecutwfc (Ry)':>12}  {'Energy (eV)':>14}  {'dE/atom (meV)':>14}  {'Status':>10}")
print("-" * 60)
ref_energy = result_ecut.converged_results[-1].energy if result_ecut.converged_results else None
for r in result_ecut.results:
    status = "converged" if r.is_converged else "FAILED"
    de = ""
    if r.energy is not None and ref_energy is not None:
        de = f"{abs(r.energy - ref_energy) / len(si) * 1000:.3f}"
    print(f"{r.parameter_value:12.1f}  {r.energy or 0:14.6f}  {de:>14}  {status:>10}")

# Convergence plot
ecut_plot_path = os.path.join(ecut_dir, "ecutwfc_convergence.png")
conv_ecut.plot(result_ecut, ecut_plot_path)
display(Image(filename=ecut_plot_path))
print(f"Saved: {ecut_plot_path}")

optimal_ecutwfc = result_ecut.converged_value or 30.0
print(f"\n>>> Optimal ecutwfc: {optimal_ecutwfc} Ry")

### 4b. K-point Convergence

Fix ecutwfc at the converged value and sweep k-point resolution
(reciprocal-space density in Bohr⁻¹). SHALOM automatically converts
resolution values to Monkhorst-Pack grids.

**Estimated time: ~5 minutes (5 SCF calculations)**

In [ ]:
from shalom.workflows import KpointConvergence

kpt_resolutions = [20, 30, 40, 50, 60]  # Bohr^-1
kpt_dir = os.path.join(OUTPUT_ROOT, "convergence_kpoints")

print(f"Testing k-point resolutions: {kpt_resolutions} Bohr\u207b\u00b9")
print(f"Fixed ecutwfc: {optimal_ecutwfc} Ry")

t0 = time.time()
conv_kpt = KpointConvergence(
    atoms=si,
    output_dir=kpt_dir,
    resolutions=kpt_resolutions,
    ecutwfc=optimal_ecutwfc,
    nprocs=NPROCS,
    wsl=WSL,
    pseudo_dir=PSEUDO_DIR,
    timeout=600,
)
result_kpt = conv_kpt.run()
elapsed = time.time() - t0

print(f"\n{result_kpt.summary()}")
print(f"Elapsed: {elapsed:.1f} s")

# Per-value results
print(f"\n{'Resolution':>10}  {'Energy (eV)':>14}  {'dE/atom (meV)':>14}  {'Status':>10}")
print("-" * 56)
ref_e = result_kpt.converged_results[-1].energy if result_kpt.converged_results else None
for r in result_kpt.results:
    status = "converged" if r.is_converged else "FAILED"
    de = ""
    if r.energy is not None and ref_e is not None:
        de = f"{abs(r.energy - ref_e) / len(si) * 1000:.3f}"
    print(f"{r.parameter_value:10.1f}  {r.energy or 0:14.6f}  {de:>14}  {status:>10}")

# Plot
kpt_plot_path = os.path.join(kpt_dir, "kpoint_resolution_convergence.png")
conv_kpt.plot(result_kpt, kpt_plot_path)
display(Image(filename=kpt_plot_path))
print(f"Saved: {kpt_plot_path}")

## 5. Full 5-Step Workflow

SHALOM's `StandardWorkflow` executes the complete electronic structure pipeline:

1. **vc-relax** — Variable-cell relaxation (optimize lattice + positions)
2. **scf** — Self-consistent field (ground state charge density)
3. **bands** — Band structure along high-symmetry k-path (seekpath)
4. **nscf** — Non-self-consistent calculation on dense k-mesh (for DOS)
5. **dos.x** — Density of states post-processing

All steps share the seekpath-standardized primitive cell for consistency.

**Estimated time: ~10–15 minutes**

In [ ]:
from shalom.workflows import StandardWorkflow

wf_dir = os.path.join(OUTPUT_ROOT, "workflow")

print("Starting 5-step QE workflow for Si...")
print(f"Output: {wf_dir}\n")

t0 = time.time()
wf = StandardWorkflow(
    atoms=si,
    output_dir=wf_dir,
    nprocs=NPROCS,
    wsl=WSL,
    pseudo_dir=PSEUDO_DIR,
    accuracy="standard",
    timeout=3600,
    dos_emin=-15.0,
    dos_emax=10.0,
)
result = wf.run()
elapsed = time.time() - t0

print(f"\n{'='*60}")
print(f"Workflow completed in {elapsed:.1f} s ({elapsed/60:.1f} min)")
print(f"{'='*60}")

# Step summary
for step in result["step_results"]:
    status = "OK" if step.success else "FAIL"
    t_str = f"({step.elapsed_seconds:.1f}s)" if step.elapsed_seconds > 0 else ""
    summ = f" \u2014 {step.summary}" if step.summary else ""
    print(f"  [{step.step_number}/5] {step.name:12s}  {status}  {t_str}{summ}")

print(f"\nFermi energy: {result['fermi_energy']:.4f} eV")
print(f"Bands plot:   {result['bands_png']}")
print(f"DOS plot:     {result['dos_png']}")

## 6. Workflow Plots (auto-generated)

The workflow automatically generates band structure and DOS plots.
Let's display them.

In [ ]:
if result["bands_png"] and os.path.exists(result["bands_png"]):
    print("Band structure (auto-generated):")
    display(Image(filename=result["bands_png"]))

if result["dos_png"] and os.path.exists(result["dos_png"]):
    print("\nDOS (auto-generated):")
    display(Image(filename=result["dos_png"]))

## 7. Electronic Structure Analysis

Quantitative analysis of the band structure:
- Band gap (eV), direct vs indirect classification
- Valence band maximum (VBM) and conduction band minimum (CBM) positions
- Effective masses via parabolic fitting near band edges
- DOS at the Fermi level

In [ ]:
from shalom.analysis import analyze_band_structure
from shalom.backends import parse_xml_bands, parse_dos_file, find_xml_path

# Parse band structure from XML
bands_dir = result["calc_dirs"]["bands"]
nscf_dir = result["calc_dirs"]["nscf"]
fermi = result["fermi_energy"]

xml_path = find_xml_path(bands_dir)
if xml_path is None:
    xml_path = find_xml_path(os.path.join(result["calc_dirs"]["scf"], "tmp"))

bs_data = parse_xml_bands(xml_path, fermi_energy=fermi)

# Parse DOS
dos_file = os.path.join(nscf_dir, "pwscf.dos")
dos_data = parse_dos_file(dos_file)
dos_data.fermi_energy = fermi

# Analyze
elec = analyze_band_structure(bs_data, dos_data=dos_data)

print("=== Electronic Structure Analysis ===")
if elec.bandgap_eV is not None:
    gap_type = "direct" if elec.is_direct else "indirect"
    print(f"Band gap:        {elec.bandgap_eV:.3f} eV ({gap_type})")
else:
    print(f"Band gap:        metallic")
print(f"Is metal:        {elec.is_metal}")
if elec.vbm_energy is not None:
    print(f"VBM energy:      {elec.vbm_energy:.4f} eV (k-index: {elec.vbm_k_index})")
if elec.cbm_energy is not None:
    print(f"CBM energy:      {elec.cbm_energy:.4f} eV (k-index: {elec.cbm_k_index})")
if elec.effective_mass_electron is not None:
    print(f"Electron m*:     {elec.effective_mass_electron:.3f} m_e")
if elec.effective_mass_hole is not None:
    print(f"Hole m*:         {elec.effective_mass_hole:.3f} m_e")
if elec.dos_at_fermi is not None:
    print(f"DOS at E_F:      {elec.dos_at_fermi:.4f} states/eV")
print(f"Occupied bands:  {elec.n_occupied_bands}")
print(f"\nNote: PBE typically underestimates Si band gap (exp: 1.17 eV)")

## 8. Combined Band Structure + DOS Plot

Publication-quality side-by-side figure with band structure on the left and
DOS on the right, sharing a common energy axis.

In [ ]:
from shalom.plotting import CombinedPlotter
from shalom.backends.qe_config import generate_band_kpath

# Re-generate k-path labels for the parsed band data
calc_atoms = result["atoms"]
kpath_cfg = generate_band_kpath(calc_atoms, npoints=40, is_2d=False)

if kpath_cfg.kpath_labels:
    cumulative_idx = 0
    label_by_idx = {}
    if kpath_cfg.kpath_points:
        for seg_idx, (_, npts) in enumerate(kpath_cfg.kpath_points):
            label = kpath_cfg.kpath_labels.get(seg_idx)
            if label:
                label_by_idx[cumulative_idx] = label
            cumulative_idx += npts
    bs_data.high_sym_labels = label_by_idx

    # Collapse discontinuity gaps
    if len(bs_data.kpath_distances) > 1:
        dist = bs_data.kpath_distances.copy()
        for k_idx in sorted(label_by_idx):
            if "|" in label_by_idx[k_idx] and k_idx + 1 < len(dist):
                gap = dist[k_idx + 1] - dist[k_idx]
                if gap > 0.0:
                    dist[k_idx + 1:] -= gap
        bs_data.kpath_distances = dist

combined_path = os.path.join(OUTPUT_ROOT, "si_combined_band_dos.png")
fig = CombinedPlotter(bs_data, dos_data).plot(
    output_path=combined_path,
    title="Si (diamond) \u2014 Electronic Structure",
    energy_window=(-8, 6),
)
plt.show()
print(f"Saved: {combined_path}")

## 9. Phonon Analysis (phonopy)

Phonon dispersion and DOS reveal dynamical stability and thermal properties.
We use the finite displacement method:

1. Generate displaced supercells (phonopy)
2. Run SCF on each displaced supercell (QE pw.x)
3. Collect forces and compute phonon properties

For Si diamond with a 2×2×2 supercell (16 atoms), symmetry reduces the
number of displacements to typically just 1.

**Estimated time: ~5 minutes**

In [ ]:
from shalom.analysis import generate_phonon_displacements, is_phonopy_available

phonon_available = is_phonopy_available()

if not phonon_available:
    print("phonopy not installed. Run: pip install phonopy")
    print("Skipping phonon analysis.")
else:
    supercell_matrix = [[2, 0, 0], [0, 2, 0], [0, 0, 2]]
    calc_atoms_phonon = result["atoms"]  # seekpath primitive cell

    displacements, ph_obj = generate_phonon_displacements(
        calc_atoms_phonon, supercell_matrix, distance=0.01
    )

    print(f"Primitive cell: {len(calc_atoms_phonon)} atoms")
    print(f"Supercell: 2\u00d72\u00d72 = {len(calc_atoms_phonon) * 8} atoms")
    print(f"Displacements needed: {len(displacements)}")

In [ ]:
# Run SCF for each displaced supercell
if phonon_available and len(displacements) > 0:
    from shalom.backends import QEBackend
    from shalom.backends.qe_config import get_qe_preset, QECalculationType
    from shalom.backends.runner import ExecutionConfig, create_runner

    phonon_dir = os.path.join(OUTPUT_ROOT, "phonon")
    os.makedirs(phonon_dir, exist_ok=True)

    force_sets = []
    backend = QEBackend()

    print(f"Running {len(displacements)} displacement SCF calculations...")
    t0 = time.time()

    for i, disp_atoms in enumerate(displacements):
        disp_dir = os.path.join(phonon_dir, f"disp_{i+1:03d}")
        os.makedirs(disp_dir, exist_ok=True)

        # Config from supercell atoms (nat must match supercell, not primitive cell)
        config = get_qe_preset(QECalculationType.SCF, atoms=disp_atoms)
        config.pseudo_dir = PSEUDO_DIR
        config.control["tprnfor"] = True  # forces needed for phonon
        config.kpoints.grid = [2, 2, 2]   # reduced k-grid for 2x2x2 supercell

        backend.write_input(disp_atoms, disp_dir, config=config)

        exec_config = ExecutionConfig(
            command="pw.x",
            input_file="pw.in",
            output_file="pw.out",
            nprocs=NPROCS,
            timeout_seconds=1800,
            wsl=WSL,
        )
        runner = create_runner(exec_config)
        exec_result = runner.run(disp_dir)

        if exec_result.success:
            dft_result = backend.parse_output(disp_dir)
            if dft_result.forces is not None:
                force_sets.append(np.array(dft_result.forces))
                print(f"  Displacement {i+1}/{len(displacements)}: "
                      f"E = {dft_result.energy:.6f} eV, "
                      f"F_max = {dft_result.forces_max:.6f} eV/\u00c5")
            else:
                print(f"  Displacement {i+1}: forces not found in output!")
        else:
            print(f"  Displacement {i+1}: FAILED (rc={exec_result.return_code})")

    elapsed = time.time() - t0
    print(f"\nPhonon SCFs completed in {elapsed:.1f} s")
    print(f"Force sets collected: {len(force_sets)}/{len(displacements)}")

In [ ]:
# Analyze phonon properties and generate plots
if phonon_available and 'force_sets' in dir() and len(force_sets) == len(displacements):
    from shalom.analysis import analyze_phonon
    from shalom.plotting import PhononBandPlotter, PhononDOSPlotter

    phonon_result = analyze_phonon(
        calc_atoms_phonon,
        supercell_matrix,
        force_sets,
        mesh=[20, 20, 20],
        band_npoints=51,
        t_max=800.0,
    )

    print("=== Phonon Analysis ===")
    print(f"Primitive cell atoms:  {phonon_result.n_atoms}")
    print(f"Phonon branches:       {phonon_result.n_branches}")
    print(f"Min frequency:         {phonon_result.min_frequency_THz:.4f} THz")
    print(f"Dynamically stable:    {phonon_result.is_stable}")
    if phonon_result.imaginary_modes:
        print(f"Imaginary modes:       {len(phonon_result.imaginary_modes)}")

    # Thermal properties at 300 K
    if phonon_result.thermal_temperatures is not None:
        idx_300 = np.argmin(np.abs(phonon_result.thermal_temperatures - 300.0))
        T = phonon_result.thermal_temperatures[idx_300]
        print(f"\nThermal properties at {T:.0f} K:")
        print(f"  Free energy:    {phonon_result.thermal_free_energy[idx_300]:.3f} kJ/mol")
        print(f"  Entropy:        {phonon_result.thermal_entropy[idx_300]:.3f} J/K/mol")
        print(f"  Heat capacity:  {phonon_result.thermal_cv[idx_300]:.3f} J/K/mol")

    # Phonon band structure
    ph_bands_path = os.path.join(OUTPUT_ROOT, "si_phonon_bands.png")
    fig = PhononBandPlotter(phonon_result).plot(
        output_path=ph_bands_path,
        title="Si \u2014 Phonon Dispersion",
    )
    plt.show()
    print(f"Saved: {ph_bands_path}")

    # Phonon DOS
    ph_dos_path = os.path.join(OUTPUT_ROOT, "si_phonon_dos.png")
    fig = PhononDOSPlotter(phonon_result).plot(
        output_path=ph_dos_path,
        title="Si \u2014 Phonon DOS",
    )
    plt.show()
    print(f"Saved: {ph_dos_path}")

    # Thermal properties plot
    fig, axes = plt.subplots(1, 3, figsize=(14, 4), dpi=150)
    T_arr = phonon_result.thermal_temperatures

    axes[0].plot(T_arr, phonon_result.thermal_free_energy, 'b-', lw=1.5)
    axes[0].set_xlabel("Temperature (K)")
    axes[0].set_ylabel("Free Energy (kJ/mol)")
    axes[0].set_title("Helmholtz Free Energy")
    axes[0].axhline(0, color='grey', lw=0.5, ls='--')

    axes[1].plot(T_arr, phonon_result.thermal_entropy, 'r-', lw=1.5)
    axes[1].set_xlabel("Temperature (K)")
    axes[1].set_ylabel("Entropy (J/K/mol)")
    axes[1].set_title("Vibrational Entropy")

    axes[2].plot(T_arr, phonon_result.thermal_cv, 'g-', lw=1.5)
    axes[2].set_xlabel("Temperature (K)")
    axes[2].set_ylabel("Cv (J/K/mol)")
    axes[2].set_title("Heat Capacity (Cv)")

    fig.suptitle("Si \u2014 Thermal Properties", fontsize=13, y=1.02)
    fig.tight_layout()
    thermal_path = os.path.join(OUTPUT_ROOT, "si_thermal_properties.png")
    fig.savefig(thermal_path, dpi=150, bbox_inches="tight")
    plt.show()
    print(f"Saved: {thermal_path}")
else:
    if phonon_available:
        print("Phonon calculation incomplete \u2014 some force sets missing.")
    else:
        print("Phonon analysis skipped (phonopy not installed).")

## 10. Summary

In [ ]:
print("=" * 60)
print("Si Complete Study \u2014 Summary of Results")
print("=" * 60)

if 'sym' in dir():
    print(f"\nStructure:")
    print(f"  Space group:     {sym.space_group_symbol} (#{sym.space_group_number})")
    print(f"  Crystal system:  {sym.crystal_system}")
    print(f"  Point group:     {sym.point_group}")

print(f"\nConvergence:")
print(f"  Optimal ecutwfc: {optimal_ecutwfc} Ry")
if result_kpt.converged_value:
    print(f"  Converged kpts:  {result_kpt.converged_value} Bohr\u207b\u00b9")

print(f"\nElectronic:")
print(f"  Fermi energy:    {fermi:.4f} eV")
if elec.bandgap_eV:
    gap_type = "direct" if elec.is_direct else "indirect"
    print(f"  Band gap:        {elec.bandgap_eV:.3f} eV ({gap_type})")
if elec.effective_mass_electron:
    print(f"  m*_e:            {elec.effective_mass_electron:.3f} m_e")
if elec.effective_mass_hole:
    print(f"  m*_h:            {elec.effective_mass_hole:.3f} m_e")

if phonon_available and 'phonon_result' in dir():
    print(f"\nPhonon:")
    print(f"  Stable:          {phonon_result.is_stable}")
    print(f"  Min freq:        {phonon_result.min_frequency_THz:.4f} THz")
    if phonon_result.thermal_temperatures is not None:
        idx_300 = np.argmin(np.abs(phonon_result.thermal_temperatures - 300.0))
        print(f"  Cv(300K):        {phonon_result.thermal_cv[idx_300]:.3f} J/K/mol")

print(f"\nAll outputs saved to: {OUTPUT_ROOT}")